In [1]:
import json
import pandas as pd
import re 
import glob 
import copy 
import os 
import nltk
import unicodedata 
from collections import Counter
from nltk.collocations import *
import numpy as np 

In [2]:
# --- Uncomment below if needed ---
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
#!python -m spacy download en_core_web_sm

In [3]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy

In [4]:
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
from bs4 import BeautifulSoup
from html import unescape

In [5]:
%%time 
### Creating reviews dataframe 
data_file = open("yelp_academic_dataset_review.json",encoding='utf-8')
data = []
for line in data_file:
    data.append(json.loads(line))
reviewsdf = pd.DataFrame(data)
data_file.close()

CPU times: total: 2min 52s
Wall time: 11min 18s


In [6]:
%%time
### Creating businesses dataframe 
data_file = open("yelp_academic_dataset_business.json",encoding='utf-8')
data = []
for line in data_file:
    data.append(json.loads(line))
businessdf = pd.DataFrame(data)
data_file.close()

CPU times: total: 28.7 s
Wall time: 3min 23s


In [7]:
reviewsdf_org = reviewsdf
businessdf_org = businessdf

In [8]:
%%time
city_and_state = businessdf[['business_id', 'city', 'state']]
reviewsdf = reviewsdf.merge(city_and_state, on='business_id')
reviewsdf

CPU times: total: 1min 5s
Wall time: 4min 44s


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,city,state
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,North Wales,PA
1,VJxlBnJmCDIy8DFG0kjSow,Iaee7y6zdSB3B-kRCo4z1w,XQfwVwDr-v0ZS3_CbbE5Xw,2.0,0,0,0,This is the second time we tried turning point...,2017-05-13 17:06:55,North Wales,PA
2,S6pQZQocMB1WHMjTRbt77A,ejFxLGqQcWNLdNByJlIhnQ,XQfwVwDr-v0ZS3_CbbE5Xw,4.0,2,0,1,The place is cute and the staff was very frien...,2017-08-08 00:58:18,North Wales,PA
3,WqgTKVqWVHDHjnjEsBvUgg,f7xa0p_1V9lx53iIGN5Sug,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,We came on a Saturday morning after waiting a ...,2017-11-19 02:20:23,North Wales,PA
4,M0wzFFb7pefOPcxeRVbLag,dCooFVCk8M1nVaQqcfTL3Q,XQfwVwDr-v0ZS3_CbbE5Xw,2.0,0,0,0,"Mediocre at best. The decor is very nice, and ...",2017-09-09 17:49:47,North Wales,PA
...,...,...,...,...,...,...,...,...,...,...,...
6990275,0gai4MzBzFCa7JsS31RRjg,xHu1jmrnv4DdJMuC8IxeRg,vI4vyi1dfG93oAiSRFDymA,1.0,2,0,0,Ridiculous wait times. I showed up right on ti...,2018-06-05 14:25:32,Paoli,PA
6990276,7MZu6bFdFoqr7MyPKqvoew,aYveEctPYcZiubXyEgLhTA,vI4vyi1dfG93oAiSRFDymA,5.0,4,1,2,Love this office. All the staff us super kind ...,2020-02-17 01:53:23,Paoli,PA
6990277,3emHuLQvuAw0l66TzSWmKA,oz-So7Kwo5tW51HrT-BgIg,vI4vyi1dfG93oAiSRFDymA,1.0,0,0,0,It was a HUGE mistake to give this place a try...,2021-09-22 16:51:46,Paoli,PA
6990278,T-adPwD-eP2LvuDINKOA9g,09zj3b4tM-xJjozvtk34wQ,vI4vyi1dfG93oAiSRFDymA,1.0,0,0,0,Friendly staff but the praise ends there. Gett...,2021-03-09 19:21:44,Paoli,PA


## Using smaller version of reviewsdf with just 100,000 businesses in California for faster processing and querying

In [14]:
reviewsdf = reviewsdf[reviewsdf['state'] == 'CA'].reset_index(drop=True)
reviewsdf

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,city,state
0,ZKvDG2sBvHVdF5oBNUOpAQ,wSTuiTk-sKNdcFyprzZAjg,B5XSoSG3SfvQGtKEGQ1tSQ,3.0,1,1,0,This easter instead of going to Lopez Lake we ...,2016-03-30 22:46:33,Santa Barbara,CA
1,9ag-bC-521T1i3IypXr4Ug,sx_cqxyWRUoOIoy_AXmLdQ,B5XSoSG3SfvQGtKEGQ1tSQ,4.0,1,0,0,I work with a college group that does camping ...,2013-09-07 21:16:36,Santa Barbara,CA
2,80p5xO3dbpd80T6vPTL8DQ,D7ct4p7uUk5t4LFSjVnO4A,B5XSoSG3SfvQGtKEGQ1tSQ,5.0,0,0,0,My SO and I love exploring new trails and so i...,2018-08-05 21:50:27,Santa Barbara,CA
3,RScubg9WIp165432dQfVHQ,r9VkVP_4d2_KMHA9ZN2c6w,B5XSoSG3SfvQGtKEGQ1tSQ,5.0,14,9,13,I can't believe there's only one review for th...,2011-07-06 23:14:29,Santa Barbara,CA
4,_JsMDKDQJ8uc6logHxA0-w,s0l9vVweCwvudj5TNGMl6w,B5XSoSG3SfvQGtKEGQ1tSQ,5.0,1,0,0,I love this area for hiking! I go there all th...,2016-04-03 03:24:46,Santa Barbara,CA
...,...,...,...,...,...,...,...,...,...,...,...
348851,cKuP5pnvbvpn4UbaccVcdg,OVYb1El25BgVhgbXb4e-KA,XPkkWyAkkPeH-PqXmBMmpg,5.0,0,0,0,Great experience with great people who are rea...,2021-06-22 21:41:07,Santa Barbara,CA
348852,dZC87sZ1f2ntOf876_DDLg,9w5PoliouEyNikdy6DizqQ,XPkkWyAkkPeH-PqXmBMmpg,5.0,0,0,0,I spent the week in Santa Barbara and had the ...,2021-06-28 23:53:44,Santa Barbara,CA
348853,NCafkcXIvK7E965my4RpQg,GmuHOkMR80oW3sAuF-UsAw,XPkkWyAkkPeH-PqXmBMmpg,5.0,0,0,0,"Last summer, I had the pleasure of experiencin...",2021-07-06 19:15:56,Santa Barbara,CA
348854,VxsNftGxRUDZcLD0Ns1wKg,VRXPOvlSydho0tYmSseKnQ,XPkkWyAkkPeH-PqXmBMmpg,5.0,0,0,0,Joseph and Teresa are super hosts. You will f...,2021-07-01 03:44:10,Santa Barbara,CA


In [55]:
reviewsdf = reviewsdf.head(30000)
reviewsdf

,review_id,text,business_id,city,state,tags
0,ZKvDG2sBvHVdF5oBNUOpAQ,This easter instead of going to Lopez Lake we ...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[lake, padres, national, maintain, dead]"
1,9ag-bC-521T1i3IypXr4Ug,I work with a college group that does camping ...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,[camping]
2,80p5xO3dbpd80T6vPTL8DQ,My SO and I love exploring new trails and so i...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[varies, comforting, balancing, remove, trails]"
3,RScubg9WIp165432dQfVHQ,I can't believe there's only one review for th...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[sandstone, peaks, national, fault, sunny]"
4,_JsMDKDQJ8uc6logHxA0-w,I love this area for hiking! I go there all th...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,[forest]
...,...,...,...,...,...,...
29995,Zk-q3XdjXNUxwHH5BS2b3A,I enjoyed a great view and delicious food at t...,fnGZspqyamRremXor0sAyg,Santa Barbara,CA,[]
29996,VMjZqDJYGHs_MSNXKhd4gw,Came here with some colleagues for a celebrato...,fnGZspqyamRremXor0sAyg,Santa Barbara,CA,"[colleagues, golf, celebratory, scenic]"
29997,lvaT-QAzAYWTqO7JWLEqYQ,This is a great place with an excellent panora...,fnGZspqyamRremXor0sAyg,Santa Barbara,CA,"[costly, golf, beverages, locale, ideal]"
29998,wJAkDH0gTqpQaKH9PVwF9g,Came for a late lunch and was hoping to get fl...,fnGZspqyamRremXor0sAyg,Santa Barbara,CA,"[links, scenery, sammy, appear, crunch]"


### Generate hashtags from reviews df

In [56]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [57]:
#Adjust mindf parameter
mindf = int(len(reviewsdf) * .001)
tfidf = TfidfVectorizer(sublinear_tf=True,
               analyzer='word',
#                max_features=2000,
               tokenizer=word_tokenize,
               stop_words=nltk.corpus.stopwords.words("english"),min_df=5)

In [58]:
text = list(reviewsdf['text'])
text = [re.sub(r'[^A-Za-z]+'," ",document) for document in text]


In [59]:
len(text)

30000

In [60]:
%%time
hashtagdf = pd.DataFrame(tfidf.fit_transform(text).toarray())

C:\Users\tonyt\anaconda3\envs\dsc80\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


CPU times: total: 7.89 s
Wall time: 9.01 s


In [61]:
hashtagdf.columns = tfidf.get_feature_names()

C:\Users\tonyt\anaconda3\envs\dsc80\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [62]:
hashtagdfsum = hashtagdf.sum()
tfidf_scores = [(i,hashtagdfsum[i]) for i in hashtagdfsum.index]
tfidf_scores.sort(key=lambda x : x[1],reverse=True)

In [63]:
##REMOVE?
lower_threshold = int(len(tfidf_scores) * .08)
upper_threshold = int(len(tfidf_scores) * .60)
scores = tfidf_scores[lower_threshold:]
tfidf_dict = {item[0] : item[1] for item in scores}

In [64]:
#Maybe add ability to get N most useful tags 
def generate_tags(text):
    tags = set()
    for s in text.split(' '):
        if s in tfidf_dict:
            tags.add(s)
    return list(tags)[:5]

In [65]:
reviewsdf['tags'] = reviewsdf['text'].apply(generate_tags)

C:\Users\tonyt\AppData\Local\Temp\ipykernel_23800\2776938788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewsdf['tags'] = reviewsdf['text'].apply(generate_tags)


In [66]:
reviewsdf.head(5)

,review_id,text,business_id,city,state,tags
0,ZKvDG2sBvHVdF5oBNUOpAQ,This easter instead of going to Lopez Lake we ...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[lake, padres, national, maintain, dead]"
1,9ag-bC-521T1i3IypXr4Ug,I work with a college group that does camping ...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[forever, camping, ten, areas, compared]"
2,80p5xO3dbpd80T6vPTL8DQ,My SO and I love exploring new trails and so i...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[rocks, remove, previous, balancing, trails]"
3,RScubg9WIp165432dQfVHQ,I can't believe there's only one review for th...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[range, national, fault, sunny, aimlessly]"
4,_JsMDKDQJ8uc6logHxA0-w,I love this area for hiking! I go there all th...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[public, forest]"


In [67]:
# trigram_measures = nltk.collocations.TrigramAssocMeasures()


In [68]:
# finder = BigramCollocationFinder.from_words()
# finder1 = finder.apply_freq_filter(1)
# finder.nbest(bigram_measures.pmi, 1500)
# #Maybe add bigrams to each string, so they end upo in freq dict
# # list(bigrams)

### Hashtag generation above needs to be improved, just temporarlily using it to continue with other parts

# Create graph data

1 ) Create  weighted edges between business and associated hashtags (edge weight is number of reviews that contained particular hashtag?)

2) Create weighted edges between hashtags that occur in the same business (or review ?) weight = num co coccurances in reviews?

Drop unecessary columns from reviewsdf 

In [69]:
reviewsdf = reviewsdf[['review_id','text','business_id','city','state','tags']]
reviewsdf

,review_id,text,business_id,city,state,tags
0,ZKvDG2sBvHVdF5oBNUOpAQ,This easter instead of going to Lopez Lake we ...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[lake, padres, national, maintain, dead]"
1,9ag-bC-521T1i3IypXr4Ug,I work with a college group that does camping ...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[forever, camping, ten, areas, compared]"
2,80p5xO3dbpd80T6vPTL8DQ,My SO and I love exploring new trails and so i...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[rocks, remove, previous, balancing, trails]"
3,RScubg9WIp165432dQfVHQ,I can't believe there's only one review for th...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[range, national, fault, sunny, aimlessly]"
4,_JsMDKDQJ8uc6logHxA0-w,I love this area for hiking! I go there all th...,B5XSoSG3SfvQGtKEGQ1tSQ,Santa Barbara,CA,"[public, forest]"
...,...,...,...,...,...,...
29995,Zk-q3XdjXNUxwHH5BS2b3A,I enjoyed a great view and delicious food at t...,fnGZspqyamRremXor0sAyg,Santa Barbara,CA,[]
29996,VMjZqDJYGHs_MSNXKhd4gw,Came here with some colleagues for a celebrato...,fnGZspqyamRremXor0sAyg,Santa Barbara,CA,"[golf, colleagues, difficult, scenic, ahi]"
29997,lvaT-QAzAYWTqO7JWLEqYQ,This is a great place with an excellent panora...,fnGZspqyamRremXor0sAyg,Santa Barbara,CA,"[locale, certainly, fare, corporate, panoramic]"
29998,wJAkDH0gTqpQaKH9PVwF9g,Came for a late lunch and was hoping to get fl...,fnGZspqyamRremXor0sAyg,Santa Barbara,CA,"[wings, links, flat, scenery, didnt]"


In [70]:
###Create graph between business and associated hashtags 
from collections import defaultdict
businessAndTagGraph = defaultdict(lambda: defaultdict(int))
tagsGraph = defaultdict(lambda: defaultdict(int))

In [71]:
%%time
for review in reviewsdf.itertuples():
    businessId = review[3]
    tags = review[6]
    for i in range(len(tags)): 
        #Update Edge weights (number of times hashtag occurs in review)
        currTag = tags[i]
        businessAndTagGraph[businessId][currTag] += 1
        #Create edges and weights for tags in same review 
        for j in range(len(tags)):
            if i != j:
                 tagsGraph[currTag][tags[j]] += 1


CPU times: total: 1.66 s
Wall time: 10.1 s


#### Convert business and tags graph into dataframe/csv format

In [72]:
#businessAndTagsdf = pd.DataFrame(columns=['business_id','hashtag','count'])
idcol = []
hashtagcol = []
countcol = []
ignore = set(['index_names','column_names','index','columns','data'])
for businessid in businessAndTagGraph.keys():
    if businessid in ignore:
        continue
    for vals in businessAndTagGraph[businessid].items():
        idcol.append(businessid)
        hashtagcol.append(vals[0])
        countcol.append(vals[1])
        #businessAndTagsdf.append({'business_id':businessid,'hashtag':vals[0],count:vals[1]})


In [73]:
businessAndTagsdf = pd.DataFrame(columns = ['business_id','hashtag','count'])
businessAndTagsdf['business_id'] = idcol
businessAndTagsdf['hashtag'] = hashtagcol
businessAndTagsdf['count'] = countcol

In [74]:
#Resulting df 
businessdf = businessdf[['business_id','name','address','city','state','latitude','longitude','stars','categories']]

In [75]:
businessAndTagsdf = businessAndTagsdf.merge(businessdf,on='business_id')
businessAndTagsdf

,business_id,hashtag,count,name,address,city,state,latitude,longitude,stars,categories
0,B5XSoSG3SfvQGtKEGQ1tSQ,lake,1,Los Padres National Forest,San Marcos Pass Rd,Santa Barbara,CA,34.597239,-119.510772,4.5,"Parks, Active Life"
1,B5XSoSG3SfvQGtKEGQ1tSQ,padres,1,Los Padres National Forest,San Marcos Pass Rd,Santa Barbara,CA,34.597239,-119.510772,4.5,"Parks, Active Life"
2,B5XSoSG3SfvQGtKEGQ1tSQ,national,5,Los Padres National Forest,San Marcos Pass Rd,Santa Barbara,CA,34.597239,-119.510772,4.5,"Parks, Active Life"
3,B5XSoSG3SfvQGtKEGQ1tSQ,maintain,1,Los Padres National Forest,San Marcos Pass Rd,Santa Barbara,CA,34.597239,-119.510772,4.5,"Parks, Active Life"
4,B5XSoSG3SfvQGtKEGQ1tSQ,dead,1,Los Padres National Forest,San Marcos Pass Rd,Santa Barbara,CA,34.597239,-119.510772,4.5,"Parks, Active Life"
...,...,...,...,...,...,...,...,...,...,...,...
72483,fnGZspqyamRremXor0sAyg,didnt,1,Frog Bar & Grill,405 Glen Annie Rd,Santa Barbara,CA,34.442938,-119.880037,4.0,"Restaurants, American (Traditional), Nightlife..."
72484,ukhkivp_eCkyCpNcujoDtQ,quoted,1,Goleta Mufflers & Brakes,"5921 Matthews St, Ste B",Goleta,CA,34.430054,-119.829281,4.0,"Auto Repair, Automotive"
72485,ukhkivp_eCkyCpNcujoDtQ,pipe,1,Goleta Mufflers & Brakes,"5921 Matthews St, Ste B",Goleta,CA,34.430054,-119.829281,4.0,"Auto Repair, Automotive"
72486,ukhkivp_eCkyCpNcujoDtQ,broken,1,Goleta Mufflers & Brakes,"5921 Matthews St, Ste B",Goleta,CA,34.430054,-119.829281,4.0,"Auto Repair, Automotive"


##### Convert tags graph to df/csv format

In [76]:
tag1 = []
tag2 = []
sharedOccurances = []
for tag in tagsGraph.keys():
    for vals in tagsGraph[tag].items():
        tag1.append(tag)
        tag2.append(vals[0])
        sharedOccurances.append(vals[1])

In [77]:
tagsDf = pd.DataFrame(columns = ['tag1','tag2','sharedOccurances'])
tagsDf['tag1'] = tag1
tagsDf['tag2'] = tag2
tagsDf['sharedOccurances']= sharedOccurances

In [78]:
tagsDf

,tag1,tag2,sharedOccurances
0,lake,padres,1
1,lake,national,1
2,lake,maintain,1
3,lake,dead,1
4,lake,seeing,1
...,...,...,...
364909,pipe,broken,1
364910,pipe,scenario,1
364911,scenario,quoted,1
364912,scenario,pipe,1


#### Save graphs to csv

In [80]:
businessAndTagsdf.to_csv('businessAndTags.csv')
tagsDf.to_csv('tagsGraph.csv')

# Import data into Neo4j

In [81]:
from neo4j import GraphDatabase
import neo4j

In [82]:
uri = "bolt://localhost:7666"
username = "neo4j"
password = "password"

def create_neo4j_session(uri, username, password):
    try:
        driver = GraphDatabase.driver(uri, auth=(username, password))
        session = driver.session()
        return session
    except Exception as e:
        print(f"Failed to create Neo4j session: {e}")
        return None

session = create_neo4j_session(uri, username, password)

In [83]:
# Deleting all existing constraints
deleting_constraints = "CALL apoc.schema.assert({}, {})"
session.run(deleting_constraints)

# Creating constraints for our graph
constraint_1 = "CREATE CONSTRAINT FOR (b:Business) REQUIRE b.id IS UNIQUE;"
constraint_2 = "CREATE CONSTRAINT FOR (t:tag) REQUIRE t.name IS UNIQUE;"

session.run(constraint_1)
session.run(constraint_2)

In [ ]:
%%time
# Creating tag nodes and SHARED_TAGS relationship between tags with shared occurences
create_tags = """
LOAD CSV WITH HEADERS FROM 'file:///tagsGraph.csv' AS row  
WITH row
CALL {
WITH row
MERGE (t1:tag {id: row.tag1})
MERGE (t2:tag {id: row.tag2})
MERGE (t1)-[r:SHARED_TAGS]-(t2)
SET r.shared_count = row.sharedOccurances
} IN TRANSACTIONS"""

# takes about 30 minutes for version of reviewsdf with 30,000 rows from just California
session.run(create_tags)

In [ ]:
%%time
# Creating Business nodes and HAS_TAG relationship between the business and every tag that business has had in their reviews
create_business = """
LOAD CSV WITH HEADERS FROM 'file:///businessAndTags.csv' AS row  
WITH row
CALL {
WITH row
MERGE (b:Business {id: row.business_id})
SET b.name =  row.name, 
    b.address = row.address,
    b.city = row.city,
    b.state = row.state,
    b.latitude = row.latitude,
    b.longitude = row.longitude,
    b.rating = row.stars,
    b.categories = split(row.categories, ', ')
MERGE (t:tag {name: row.hashtag})
MERGE (b)-[r:HAS_TAG]->(t)
SET r.tag_count = row.count
} IN TRANSACTIONS
"""

session.run(create_business)